# 更复杂的工作流--天气查询

In [12]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pocketflow import Node,Flow

load_dotenv()

True

In [2]:
def call_llm(prompt):
    client = OpenAI(
        base_url= os.getenv("BASE_URL"),
        api_key=os.getenv("DASHSCOPE_API_KEY")
    )
    res = client.chat.completions.create(
        model = os.getenv("MODEL_NAME"),
        messages = [{
            "role": "user",
            "content": prompt
        }]
    )
    return res.choices[0].message.content

In [3]:
# 定义收集天气节点
class CollectWeatherData(Node):
    def prep(self,shared):
        return shared['city']
    def exec(self,city):
        prompt = f"模拟API请求：请提供{city}当前的天气信息，包括温度、湿度、风速和天气状况"
        res = call_llm(prompt)
        return res
    def post(self,shared,prep_res,exec_res):
        shared['weather_data'] = exec_res
        print("\n===== 已收集到的天气数据 =====\n") 

In [8]:
# 定义分析天气节点
class AnalyzeWeatherData(Node):
    def prep(self,shared):
        return shared['weather_data']
    def exec(self,weather_data):
        prompt = f"""
        分析以下天气数据，并以JSON格式返回关键信息：  
          
        {weather_data}  
          
        请以如下格式返回：  
        ```json  
        {{  
            "temperature": "温度数值和单位",  
            "humidity": "湿度百分比",   
            "wind": "风速和风向",  
            "condition": "天气状况",  
            "summary": "一句话总结天气情况"  
        }}  
        ```"""
        res = call_llm(prompt)
        json_str = res.split("```json")[1].split("```")[0].strip()
        return json_str
    
    def post(self,shared,prep_res,exec_res):
        shared['weather_analysis'] = exec_res
        print("\n===== 已获取天气数据分析结果 =====\n")

In [9]:
# 定义生成天气预报节点
class GenerateWeatherData(Node):
    def prep(self,shared):
        return shared['weather_analysis'],shared['city']
    def exec(self,data):
        analysis,city = data
        prompt = f"""
        基于以下天气分析数据，为{city}创建一个用户友好的天气报告：  
            
        {analysis}  
            
        报告应该：  
        - 使用日常用语，避免专业术语  
        - 添加一些关于适合的活动的建议  
        - 提供简单的着装建议  
        - 采用轻松愉快的语调  
        """
        return call_llm(prompt)
    def post(self,shared,prep_res,exec_res):
        shared['weather_report'] = exec_res
        print("\n===== 已生成的天气报告 =====\n")  

In [10]:
# 创建全部节点
collect_node = CollectWeatherData()
analze_node = AnalyzeWeatherData()
report_node = GenerateWeatherData()

# 连接节点
collect_node >> analze_node >> report_node
weather_flow = Flow(start=collect_node)

In [15]:
def main():
    shared = {"city": "深圳"}
    weather_flow.run(shared)

    print("\n=== 工作流完成 ===\n")
    print(f"天气报告: {shared['weather_report']}")

if __name__ == '__main__':
    main()


===== 已收集到的天气数据 =====


===== 已获取天气数据分析结果 =====


===== 已生成的天气报告 =====


=== 工作流完成 ===

天气报告: 🌞 **深圳今日天气报告 | 轻松又舒适的多云天** 🌤️

嗨！一起来看看今天的天气吧～

今天的深圳是**多云天**，阳光偶尔露个脸，整体感觉非常舒服！气温刚刚好，**28°C**，不冷不热，正是出门溜达的好时候！

🌬️ **风力情况**：微风拂面，**风速12 km/h**，不会太晒也不会太闷，户外活动刚刚好～

💧 **空气有点湿润**：空气湿度是**65%**，体感清爽，不会觉得黏糊糊的。

---

👕 **穿什么好？**  
今天穿得**轻便舒适**就行～短袖、T恤、薄裙子都可以安排上！如果傍晚出门，可以带一件薄外套，以防微风有点凉。

---

🏃‍♀️ **适合做什么？**  
这样的天气，不出门都可惜啦！

- 公园散步、骑行超舒服  
- 去海边吹吹风（记得防晒哦）  
- 约朋友户外咖啡坐坐，享受慢时光  
- 晚上还可以去夜市逛一逛，风不大刚刚好！

---

✨ **一句话总结**：  
深圳今天多云舒适，温度刚刚好，风也温柔，快趁着好天气，安排一场轻松的小约会或户外活动吧！

🌤️ 愿你今天心情也像天气一样明媚！
